In [1]:
!pwd

/home/jupyter/siim


In [2]:
%cp /home/jhunjhunwala_uday/train.csv .

In [3]:
!ls

train.csv  Untitled.ipynb


In [6]:
!pip install pytorch_lightning > /dev/null
!pip install albumentations
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from PIL import Image
from sklearn.model_selection import StratifiedKFold
import pytorch_lightning as pl
np.random.seed(10)
torch.manual_seed(300)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import sys

     |████████████████████████████████| 117 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 948 kB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 36.7 MB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 49.5 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 94.2 MB/s eta 0:00:01
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65167 sha256=ece4cd0144a9317e1634d324e57b71d4add1c909c687a77779a4ed64e3abb0e2
  Stored in directory: /home/jupyter/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations


In [7]:
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
import torchvision.transforms as transforms
from fastai.vision import *
import PIL
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from pytorch_lightning.loggers import TensorBoardLogger
import albumentations
from torch.optim.lr_scheduler import OneCycleLR

In [8]:
torch.cuda.device(0)
use_cuda=torch.cuda.is_available()
device=torch.device('cuda:0' if use_cuda else 'cpu')

In [9]:
torch.cuda.is_available()

True

In [12]:
!sudo mv "/home/jhunjhunwala_uday/siic-isic-224x224-images.zip" .

In [15]:
MAIN_PATH='siim-isic-melanoma-classification/'
train_file='train.csv'
train_images='train/'
train_images_384='jpeg-melanoma-384x384/train/'
train_images_512='jpeg-melanoma-512x512/train/'

In [16]:
train_data=pd.read_csv(train_file)
train_data[:3]

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0


In [17]:
class_m=train_data['target'].sum()
class_b=train_data['target'].count()-class_m
class_weights=torch.tensor([1.0/class_b,1.0/class_m]).float().to(device)
class_weights

tensor([3.0730e-05, 1.7123e-03], device='cuda:0')

In [18]:
dummies=pd.get_dummies(train_data['anatom_site_general_challenge'],dummy_na=True,dtype=np.uint8, prefix='site')
dummies

,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,1,0,0,0,0,0,0
1,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0
4,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...
33121,0,0,0,0,1,0,0
33122,0,0,0,0,1,0,0
33123,0,1,0,0,0,0,0
33124,0,1,0,0,0,0,0


In [19]:
train_data=pd.concat([train_data,dummies.iloc[:train_data.shape[0]]],axis=1)
train_data

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,1,0,0,0,0,0,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0,0,1,0,0,0,0,0
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0,1,0,0,0,0,0,0
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33121,ISIC_9999134,IP_6526534,male,50.0,torso,unknown,benign,0,0,0,0,0,1,0,0
33122,ISIC_9999320,IP_3650745,male,65.0,torso,unknown,benign,0,0,0,0,0,1,0,0
33123,ISIC_9999515,IP_2026598,male,20.0,lower extremity,unknown,benign,0,0,1,0,0,0,0,0
33124,ISIC_9999666,IP_7702038,male,50.0,lower extremity,unknown,benign,0,0,1,0,0,0,0,0


In [20]:
train_data['sex']=train_data['sex'].map({'male':1,'female':0})
train_data[:3]

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,ISIC_2637011,IP_7279968,1.0,45.0,head/neck,unknown,benign,0,1,0,0,0,0,0,0
1,ISIC_0015719,IP_3075186,0.0,45.0,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
2,ISIC_0052212,IP_2842074,0.0,50.0,lower extremity,nevus,benign,0,0,1,0,0,0,0,0


In [21]:
train_data['sex']=train_data['sex'].fillna(-1)
train_data[:3]

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,ISIC_2637011,IP_7279968,1.0,45.0,head/neck,unknown,benign,0,1,0,0,0,0,0,0
1,ISIC_0015719,IP_3075186,0.0,45.0,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
2,ISIC_0052212,IP_2842074,0.0,50.0,lower extremity,nevus,benign,0,0,1,0,0,0,0,0


In [22]:
train_data['age_approx']/=90
train_data[:3]

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,ISIC_2637011,IP_7279968,1.0,0.500000,head/neck,unknown,benign,0,1,0,0,0,0,0,0
1,ISIC_0015719,IP_3075186,0.0,0.500000,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
2,ISIC_0052212,IP_2842074,0.0,0.555556,lower extremity,nevus,benign,0,0,1,0,0,0,0,0


In [23]:
train_data['age_approx']=train_data['age_approx'].fillna(0)
train_data[:3]

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,site_head/neck,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_upper extremity,site_nan
0,ISIC_2637011,IP_7279968,1.0,0.500000,head/neck,unknown,benign,0,1,0,0,0,0,0,0
1,ISIC_0015719,IP_3075186,0.0,0.500000,upper extremity,unknown,benign,0,0,0,0,0,0,1,0
2,ISIC_0052212,IP_2842074,0.0,0.555556,lower extremity,nevus,benign,0,0,1,0,0,0,0,0


In [24]:
meta_features=['sex','age_approx']+[col for col in train_data.columns if col.startswith('site_')]
n_meta_features=len(meta_features)
meta_features

['sex',
 'age_approx',
 'site_head/neck',
 'site_lower extremity',
 'site_oral/genital',
 'site_palms/soles',
 'site_torso',
 'site_upper extremity',
 'site_nan']

In [25]:
image_size=224
train_transforms=albumentations.Compose([
    albumentations.Transpose(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.RandomBrightness(limit=0.2, p=0.75),
    albumentations.RandomContrast(limit=0.2, p=0.75),
    albumentations.OneOf([
        albumentations.MotionBlur(blur_limit=5),
        albumentations.MedianBlur(blur_limit=5),
        albumentations.GaussianBlur(blur_limit=5),
        albumentations.GaussNoise(var_limit=(5.0, 30.0)),
        ], p=0.7),

    albumentations.OneOf([
        albumentations.OpticalDistortion(distort_limit=1.0),
        albumentations.GridDistortion(num_steps=5, distort_limit=1.),
        albumentations.ElasticTransform(alpha=3),
        ], p=0.7),
    albumentations.CLAHE(clip_limit=4.0, p=0.7),
    albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
    albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    albumentations.Cutout(max_h_size=int( image_size* 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
    albumentations.Normalize()
    
])

valid_transforms=albumentations.Compose([
    albumentations.Normalize()
    
])


In [26]:
class MelanomaDataset(Dataset):
    def __init__(self,df,idxs,transforms):
        if idxs is None:
            self.df=df
            return
        self.df=df.loc[idxs].reset_index(drop=True)
        self.transforms=transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        #pil2tensor=transforms.ToTensor()
        #img=pil2tensor(Image.open(train_images+self.df.loc[idx,'image_name']+'.png'))
        img=PIL.Image.open(train_images+self.df.loc[idx,'image_name']+'.png')
        img=np.array(img)
        img=self.transforms(image=img)['image']
        img=img.transpose(2,0,1)
        img=torch.tensor(img).float()
        meta_data=torch.tensor(self.df.iloc[idx][meta_features]).float()
        return ((img,meta_data),self.df.loc[idx,'target'])
    def get_targets(self):
        targets=list(self.df['target'])
        return targets

In [27]:
y=train_data['target'].to_numpy()
x=np.zeros(33126)
skf=StratifiedKFold(n_splits=5)
train=[]
valid=[]
counter=0
for train_idx,test_idx in skf.split(x,y):
    train.append(train_idx)
    valid.append(test_idx)
    counter+=1
    total=0
    for i in test_idx:
        total+=y[i]
    print(total)

117
116
117
117
117


In [28]:
train_dataset=MelanomaDataset(train_data,train[0],train_transforms)
valid_dataset=MelanomaDataset(train_data,valid[0],valid_transforms)
print(len(train_dataset))
print(len(valid_dataset))

26500
6626


In [29]:
label_weights=[class_weights[x] for x in train_dataset.get_targets()]
print(len(label_weights))
#print(label_weights)

26500


In [30]:
total_dataset_size=26500
sampler = WeightedRandomSampler(label_weights,total_dataset_size)

In [31]:
train_dataloader=torch.utils.data.DataLoader(train_dataset,shuffle=False,batch_size=256,sampler=sampler,
                                             pin_memory=True,num_workers=4)

In [32]:
valid_dataloader=DataLoader(valid_dataset,shuffle=False,batch_size=256,pin_memory=True,num_workers=4,drop_last=True)

In [38]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

class MelanomaModel(nn.Module):
    def __init__(self):
        super().__init__()
        num_target_classes=2
        self.classifier = EfficientNet.from_pretrained('efficientnet-b2')
        self.bn1=nn.BatchNorm2d(1408,eps=1e-3,momentum=0.01)
        self.adaptive=nn.AdaptiveAvgPool2d(output_size=1)
        #self.classifier.eval()
        #self.classifier.avgpool=AdaptiveConcatPool2d(output_size=1)
        '''actual_layer1=nn.Sequential(
                                self.classifier.conv1,
                                self.classifier.bn1,
                                self.classifier.relu,
                                self.classifier.maxpool,
                                self.classifier.layer1,
                                self.classifier.layer2,
                                self.classifier.layer3,
                                self.classifier.layer4,
                                self.classifier.avgpool
                            )
      
        fc1=nn.Sequential(
                            nn.Flatten(),
                            nn.BatchNorm1d(1024),
                            nn.Dropout(p=0.25),
                            nn.Linear(1024,512),
                            nn.ReLU(),
                            nn.BatchNorm1d(512))
        fc=nn.Sequential(
                            nn.Flatten(),
                            nn.BatchNorm1d(1024),
                            nn.Dropout(p=0.25),
                            nn.Linear(1024,512),
                            nn.ReLU(),
                            nn.BatchNorm1d(512),
                            nn.Dropout(p=0.5),
                            nn.Linear(512, num_target_classes))'''
        
        #self.classifier=nn.Sequential(self.classifier, big)
        self.meta=nn.Sequential(
                                nn.Linear(n_meta_features, 512),
                                nn.ReLU(),
                                nn.BatchNorm1d(512),
                                nn.Dropout(p=0.3),
                                nn.Linear(512,256),
                                nn.ReLU(),
                                nn.BatchNorm1d(256))
                                
        self.final_layer=nn.Sequential(
                                        nn.Linear(1664,256),
                                        nn.ReLU(),
                                        nn.BatchNorm1d(256),
                                        nn.Dropout(p=0.3),
                                        nn.Linear(256,2))
        
        #self.updateable_params=[]
        #for p in self.classifier.fc.parameters():
        #    self.updateable_params.append(p)
        # use the pretrained model to classify cifar-10 (10 image classes)
        
    def forward(self,x,x_meta):
        x=self.classifier.extract_features(x)
        x=self.bn1(x)
        x=self.adaptive(x)
        x=x.squeeze(-1).squeeze(-1)
        #print(x.shape)
        #x=nn.Flatten(x)
        #x1=self.classifier(x)
        x_meta=self.meta(x_meta)
        #print(x_meta.shape)
        x=torch.cat((x,x_meta),dim=1)
        return self.final_layer(x)

In [34]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=16034 sha256=47c5de74cdea366433cabb833ab438f3444789d755d1a9e4c2e60f1c3e5a14df
  Stored in directory: /home/jupyter/.cache/pip/wheels/b7/cc/0d/41d384b0071c6f46e542aded5f8571700ace4f1eb3f1591c29
Successfully built efficientnet-pytorch


In [39]:
model=MelanomaModel()
model.to(device)

Loaded pretrained weights for efficientnet-b2


MelanomaModel(
  (classifier): EfficientNet(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
          (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        )
        (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          32, 8, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          8, 32, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
     

In [40]:
data=DataBunch(train_dataloader, valid_dataloader,device=device)

In [41]:
learn=Learner(data,model,loss_func=F.cross_entropy, metrics=[error_rate,accuracy,AUROC()])

In [42]:
for p in learn.model.classifier.parameters():
    p.requires_grad_(False)

In [43]:
counter=0
total_params=0
for p in learn.model.parameters():
    total_params+=1
    if p.requires_grad:
        counter+=1
print(counter)
print(total_params)

16
317


In [44]:
learn.model.to(device)
learn.data.device

device(type='cuda', index=0)

In [45]:
counter=0
for p in learn.model.parameters():
    if p.requires_grad:
        counter+=1
print(counter)

16


In [46]:
learn.fit_one_cycle(2,max_lr=1e-3)

epoch,train_loss,valid_loss,error_rate,accuracy,auroc,time
0,0.572475,0.236053,0.066406,0.933594,0.801801,04:24
1,0.523997,0.363957,0.165000,0.835000,0.827466,04:22
